### Jul AdventKalender D23

https://adventofcode.com/2023/day/23

#### Part 1 

In [1]:
with open("data/input23.txt","r") as file:
    map_input = [list(line.strip()) for line in file.readlines()]

In [2]:
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)] # right, down, left, up
slide_directions = {'>':0, 'v':1, '<':2, '^':3}

def is_valid_move(pos):
    global map_input
    n_rows, n_cols = len(map_input), len(map_input[0])
    return (0 <= pos[0] < n_rows) and (0 <= pos[1] < n_cols) and (map_input[pos[0]][pos[1]] != '#')

def get_neighbors(pos):
    global directions, map_input
    res = []
    if map_input[pos[0]][pos[1]] in slide_directions:
        dir_must = directions[slide_directions[map_input[pos[0]][pos[1]]]]
        res.append((pos[0] + dir_must[0], pos[1] + dir_must[1]))
    else:
        for direction in directions:
            next_pos = (pos[0] + direction[0], pos[1] + direction[1])
            if is_valid_move(next_pos):
                res.append(next_pos)
    return res

def get_longest_path(start_pos, end_pos):
    global map_input
    todolist = [(start_pos, set())] # reached pos, path took to reach here
    visited = {start_pos:0} # reached pos: longest steps took to reach here
    while todolist:
        cur_pos, path = todolist.pop(0)
        if cur_pos != end_pos:
            neighbors = get_neighbors(cur_pos)
            for neighbor in neighbors:
                n_steps = visited[cur_pos] + 1
                if (neighbor not in path) and ((neighbor not in visited) or (n_steps > visited[neighbor])):
                    visited[neighbor] = n_steps
                    next_path = path.copy()
                    next_path.add(neighbor)
                    todolist.append((neighbor, next_path))

    return visited[end_pos]
get_longest_path((0,1), (len(map_input)-1,len(map_input[0])-2))

2202

#### Part 2

In [3]:
map_input = []
with open("data/input23.txt","r") as file:
    for line in file.readlines():
        line = line.strip().replace('>','.').replace('v','.').replace('<','.').replace('^','.')
        map_input.append(list(line))

[collections vs. list](https://stackoverflow.com/questions/23487307/python-deque-vs-list-performance-comparison)

In [5]:
import collections # for faster running, compared to normal list pop(0) and append

def get_path_splits(start_pos, end_pos): # find path between splits. no going back and no splits in between -> bfs
    global map_input, splits
    todolist = [start_pos]
    visited = {start_pos:0}
    while todolist:
        cur_pos = todolist.pop(0)
        if cur_pos == end_pos:
            break
        if cur_pos != start_pos and (cur_pos in splits): # come across a split, discard
            continue
        neighbors = get_neighbors(cur_pos)
        for neighbor in neighbors:
            if neighbor not in visited:
                visited[neighbor] = visited[cur_pos] + 1
                todolist.append(neighbor)

    return visited.get(end_pos) # sometimes cannot find path (when there are absolut other splits in between)

def get_path_regions(start_split, end_split): # find longest path between regions
    global splits_steps
    todolist = collections.deque([(start_split, 0, set())])
    res = 0
    while todolist:
        cur_split, n_steps, path = todolist.pop()
        if cur_split == end_split:
            res = max(res, n_steps)
            continue
        next_path = path.copy()
        next_path.add(cur_split)
        for next_split,next_split_steps in splits_steps[cur_split].items(): # all splits the current split connect
            if (next_split not in path):
                todolist.appendleft((next_split, n_steps+next_split_steps, next_path))
    return res

In [6]:
start_pos, end_pos = (0,1), (len(map_input)-1,len(map_input[0])-2)

splits = [] # positions that brach to more than 1 path
splits_steps={start_pos:{}, end_pos:{}} # steps from a split to another split, including start and end
for i in range(len(map_input)):
    for j in range(len(map_input[0])):
        if map_input[i][j] != '#':
            neighbors = get_neighbors((i,j))
            if len(neighbors)>2:
                splits.append((i,j))
                splits_steps[(i,j)] = {}
                
for split_from in splits_steps.keys():
    for split_end in splits:
        if split_from != split_end:
            # get path from the two splits
            n_steps = get_path_splits(split_from, split_end)
            if n_steps is not None:
                splits_steps[split_from][split_end] = splits_steps[split_end][split_from] = n_steps

get_path_regions(start_pos, end_pos)

6226